# Datasets库使用

Datasets支持csv,text,json,pandas本地文件，也支持远程下载

In [110]:
# 下载本地数据集 并手动解压缩
!wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-train.json.gz
!wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-test.json.gz

'wget' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
'wget' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


In [111]:
# 加载
from datasets import load_dataset

dataset = load_dataset('json',data_files='./SQuAD_it-train.json',field='data')

In [112]:
print(dataset['train'][0])

{'title': 'Terremoto del Sichuan del 2008', 'paragraphs': [{'context': "Il terremoto del Sichuan del 2008 o il terremoto del Gran Sichuan, misurato a 8.0 Ms e 7.9 Mw, e si è verificato alle 02:28:01 PM China Standard Time all' epicentro (06:28:01 UTC) il 12 maggio nella provincia del Sichuan, ha ucciso 69.197 persone e lasciato 18.222 dispersi.", 'qas': [{'answers': [{'answer_start': 29, 'text': '2008'}], 'id': '56cdca7862d2951400fa6826', 'question': 'In quale anno si è verificato il terremoto nel Sichuan?'}, {'answers': [{'answer_start': 232, 'text': '69.197'}], 'id': '56cdca7862d2951400fa6828', 'question': 'Quante persone sono state uccise come risultato?'}, {'answers': [{'answer_start': 29, 'text': '2008'}], 'id': '56d4f9902ccc5a1400d833c0', 'question': 'Quale anno ha avuto luogo il terremoto del Sichuan?'}, {'answers': [{'answer_start': 78, 'text': '8.0 Ms e 7.9 Mw'}], 'id': '56d4f9902ccc5a1400d833c1', 'question': 'Che cosa ha fatto la misura di sisma?'}, {'answers': [{'answer_star

In [113]:
# 直接使用load_data加载的只有train标签
# 如果想要train和test都在需要配置
# load_dataset里配置的data_files，本地文件就算是gz和zip等都可以自动解压
data_files = {"train":"./SQuAD_it-train.json","test":"./SQuAD_it-test.json"}
dataset = load_dataset('json',data_files=data_files,field='data')

In [114]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 442
    })
    test: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 48
    })
})


In [115]:
# # 加载远程数据集
# url = "https://github.com/crux82/squad-it/raw/master/"
# data_files = {
#     "train": url + "SQuAD_it-train.json.gz",
#     "test": url + "SQuAD_it-test.json.gz",
# }
# dataset = load_dataset('json',data_files=data_files,field='data')

# 数据操作

In [116]:
# !wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip"
# !unzip drugsCom_raw.zip

In [117]:
# 使用托管在 加州大学欧文分校机器学习仓库 的 药物审查数据集 ，其中包含患者对各种药物的评论，以及正在治疗的病情和患者满意度的 10 星评价
#  TSV 仅仅是 CSV 的一个变体，它使用制表符而不是逗号作为分隔符，我们可以使用加载 csv 文件的 load_dataset() 函数并指定分隔符，来加载这些文件
from datasets import load_dataset
data_files = {
    "train":"./drugsCom_raw/drugsComTrain_raw.tsv",
    "test":"./drugsCom_raw/drugsComTest_raw.tsv",
}
dataset = load_dataset('csv',data_files=data_files,delimiter='\t')

In [118]:
samples = dataset['train'].shuffle(seed=42).select(range(1000)) # 随机选择1000个样本
print(len(samples))
print(samples[:2])

1000
{'Unnamed: 0': [87571, 178045], 'drugName': ['Naproxen', 'Duloxetine'], 'condition': ['Gout, Acute', 'ibromyalgia'], 'review': ['"like the previous person mention, I&#039;m a strong believer of aleve, it works faster for my gout than the prescription meds I take. No more going to the doctor for refills.....Aleve works!"', '"I have taken Cymbalta for about a year and a half for fibromyalgia pain. It is great\r\nas a pain reducer and an anti-depressant, however, the side effects outweighed \r\nany benefit I got from it. I had trouble with restlessness, being tired constantly,\r\ndizziness, dry mouth, numbness and tingling in my feet, and horrible sweating. I am\r\nbeing weaned off of it now. Went from 60 mg to 30mg and now to 15 mg. I will be\r\noff completely in about a week. The fibro pain is coming back, but I would rather deal with it than the side effects."'], 'rating': [9.0, 3.0], 'date': ['September 2, 2015', 'November 7, 2011'], 'usefulCount': [36, 13]}


In [119]:
# 有个字段叫  Unnamed:0 猜测是用户id，可以看下Dataset.unique() 函数来验证匿名 ID 的数量是否与分割后每个分组中的行数匹配

for split in dataset.keys():
    assert len(dataset[split]) == len(dataset[split].unique('Unnamed: 0')) # 数量一致  unique() 用于获取某一列的唯一值

In [120]:
dataset = dataset.rename_column('Unnamed: 0','patient_id')
dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 161297
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53766
    })
})

In [121]:
#  使用 Dataset.unique() 函数查找训练和测试集中的特定药物和病症的数量。
len(dataset['train'].unique('drugName'))

3436

In [122]:
# 使用map函数，规范所有病症名字，都转为小写
def lowercase_condition(example):
    return {"condition":example['condition'].lower()}
def filter_nones(x):
    return x['condition'] is not None # 可以写成lambda形式
print(dataset['train']['condition'][:10])
dataset = dataset.filter(lambda x: x['condition'] is not None)
dataset = dataset.map(lowercase_condition)
print(dataset['train']['condition'][:10])


['Left Ventricular Dysfunction', 'ADHD', 'Birth Control', 'Birth Control', 'Opiate Dependence', 'Benign Prostatic Hyperplasia', 'Emergency Contraception', 'Bipolar Disorde', 'Epilepsy', 'Birth Control']
['left ventricular dysfunction', 'adhd', 'birth control', 'birth control', 'opiate dependence', 'benign prostatic hyperplasia', 'emergency contraception', 'bipolar disorde', 'epilepsy', 'birth control']


In [123]:
# 统计数据中用户的评论字数
def compute_review_length(example):
    return {"review_length":len(example['review'].split())}
dataset = dataset.map(compute_review_length)

print(dataset['train']['review_length'][:10])

[17, 141, 134, 89, 124, 68, 30, 146, 35, 142]


In [124]:
# 看看最长评论多长
dataset['train'].sort('review_length',reverse=True)[:3]
# 看看最短的多长
dataset['train'].sort('review_length')[:3]


{'patient_id': [111469, 13653, 53602],
 'drugName': ['Ledipasvir / sofosbuvir',
  'Amphetamine / dextroamphetamine',
  'Alesse'],
 'condition': ['hepatitis c', 'adhd', 'birth control'],
 'review': ['"Headache"', '"Great"', '"Awesome"'],
 'rating': [10.0, 10.0, 10.0],
 'date': ['February 3, 2015', 'October 20, 2009', 'November 23, 2015'],
 'usefulCount': [41, 3, 0],
 'review_length': [1, 1, 1]}

In [125]:
# 太短没法判断情感分析了 过滤掉评论数量小于30个的样本

dataset = dataset.filter(lambda x:x['review_length']>30)
dataset['train'].sort('review_length')[:3]

{'patient_id': [208641, 118552, 2448],
 'drugName': ['Amlodipine / olmesartan',
  'Amoxicillin / clarithromycin / lansoprazole',
  'Emend'],
 'condition': ['high blood pressure',
  'helicobacter pylori infection',
  'nausea/vomiting, postoperative'],
 'review': ['"My blood pressure has been around 160/100. Doctor prescribed Azor 40/10. Just 4 hrs later my reading showed 120/82. I was amazed. I am now on it daily. Thanks to Azor."',
  '"I had severe vomiting and diarhoea for 3 days caused by clarythromycin. After being treated for dehydration at the hospital, clarythormycin was replaced with doxycycline, and I have no problems since."',
  '"I always get nausea and vomiting with anesthesia even when taking other anti-nausea meds. Was given Emend prior to gallbladder removal. Woke up with absolutely no nausea. Worked great for me!"'],
 'rating': [10.0, 2.0, 10.0],
 'date': ['January 19, 2015', 'February 18, 2017', 'August 3, 2016'],
 'usefulCount': [10, 4, 1],
 'review_length': [31, 31, 3

In [126]:
dataset.num_rows # 可以看到删除了15%的样本了

# 处理评论中的html字符，因为是网页爬虫爬下来的字符，有些字符需要用html库解码
import html
text = "I&#039;m a transformer called BERT"
html.unescape(text)

"I'm a transformer called BERT"

In [127]:
dataset = dataset.map(lambda e: {"review":html.unescape(e['review'])})
dataset['train']['review'][:2]

['"My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. Overall his behavior is better. \r\nWe have tried many different medications and so far this is the most effective."',
 '"I used to take another oral contraceptive, which had 21 pill cycle, and was very happy- very light periods, max 5 days, no other side effects. But it contained hormone gestodene, which is not available in US, so I switched to Lybrel, because the ing

In [128]:
# map方法执行比较慢，可以用batched=True来加速 同时处理多个元素来加速
dataset = dataset.map(
    lambda e: {"review":[html.unescape(o) for o in e['review']]}  # 一旦batched=True，e就不是单个样本，是一批次样本，需要遍历了
    ,batched=True
    )


In [129]:
dataset['train']['review'][:2]

['"My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. Overall his behavior is better. \r\nWe have tried many different medications and so far this is the most effective."',
 '"I used to take another oral contraceptive, which had 21 pill cycle, and was very happy- very light periods, max 5 days, no other side effects. But it contained hormone gestodene, which is not available in US, so I switched to Lybrel, because the ing

In [131]:
from transformers import AutoTokenizer
# tokenizer如果设置use_fast = False 那么不会自动批量多线程执行，会慢
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased',use_fast=True)
# tokenize化并且最大截断长度为128
def tokenize_and_split(example):
    return tokenizer(
        example['review']
        ,truncation=True
        ,max_length=128
        ,return_overflowing_tokens=True  # 这个设置开启后，如果评论超过了128，会被截成多段，每一段都会返回，而不是只返回第一段
    )
#测试
tokenize_and_split(dataset['train'][:1])['input_ids']

[[101,
  107,
  1422,
  1488,
  1110,
  9079,
  1194,
  1117,
  2223,
  1989,
  1104,
  1130,
  19972,
  11083,
  119,
  1284,
  1245,
  4264,
  1165,
  1119,
  1310,
  1142,
  1314,
  1989,
  117,
  1165,
  1119,
  1408,
  1781,
  1103,
  2439,
  13753,
  1119,
  1209,
  1129,
  1113,
  119,
  1370,
  1160,
  1552,
  117,
  1119,
  1180,
  6374,
  1243,
  1149,
  1104,
  1908,
  117,
  1108,
  1304,
  172,
  14687,
  1183,
  117,
  1105,
  7362,
  1111,
  2212,
  129,
  2005,
  1113,
  170,
  2797,
  1313,
  1121,
  1278,
  12020,
  113,
  1304,
  5283,
  1111,
  1140,
  119,
  114,
  146,
  1270,
  1117,
  3995,
  1113,
  6356,
  2106,
  1105,
  1131,
  1163,
  1106,
  6166,
  1122,
  1149,
  170,
  1374,
  1552,
  119,
  3969,
  1293,
  1119,
  1225,
  1120,
  1278,
  117,
  1105,
  1114,
  2033,
  1146,
  1107,
  1103,
  2106,
  119,
  1109,
  1314,
  1160,
  1552,
  1138,
  1151,
  2463,
  1714,
  119,
  1124,
  1110,
  150,
  21986,
  3048,
  1167,
  5340,
  1895,
  1190,
  1518,

In [101]:
tokenized_dataset = dataset.map(tokenize_and_split,batched=True,remove_columns=dataset['train'].column_names)
 # 当使用 batched=True 和 return_overflowing_tokens=True 时，某些批次可能生成更多的样本（例如，100 个输入样本可能生成 146 个输出片段
 # Datasets 库要求新生成的列必须与原始列长度一致，因此报错 ArrowInvalid: Column length mismatch
#  batched=False 时 return_overflowing_tokens=True 仍然会拆分长文本，但返回的是一个包含多个片段的列表
# remove_columns=dataset['train'].column_names 删除所有原始列就好了

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

In [102]:
print(tokenized_dataset)
print(len(dataset['train']))
print(tokenized_dataset['train']['overflow_to_sample_mapping'][:10])

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping'],
        num_rows: 206772
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping'],
        num_rows: 68876
    })
})
138514
[0, 0, 1, 1, 2, 3, 3, 4, 5, 5]


In [103]:
# tokenized_dataset删除了所有旧的字段，导致labels也没了，所以需要修改为添加手动匹配映射的版本
# overflow_to_sample_mapping是句子对应的原始集合内的序号
def tokenize_and_split(examples):
    result = tokenizer(
        examples['review']
        ,truncation=True
        ,max_length=128
        ,return_overflowing_tokens=True
    )
    # 提取映射
    sample_map = result.pop('overflow_to_sample_mapping') # 整个列pop出去 sample_map的值是一个列所有[..,..,..,..]
    for key,values in examples.items():
        result[key] = [values[i] for i in sample_map]
    return result


In [104]:
tokenized_dataset = dataset.map(tokenize_and_split,batched=True)

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

In [105]:
test = tokenized_dataset.map(lambda x: {"inputs_ids_length":[len(o) for o in x['input_ids']]},batched=True)
test.sort('inputs_ids_length')
test.filter(lambda x:x['inputs_ids_length']>30)

Map:   0%|          | 0/206772 [00:00<?, ? examples/s]

Map:   0%|          | 0/68876 [00:00<?, ? examples/s]

Filter:   0%|          | 0/206772 [00:00<?, ? examples/s]

Filter:   0%|          | 0/68876 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask', 'inputs_ids_length'],
        num_rows: 187032
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask', 'inputs_ids_length'],
        num_rows: 62224
    })
})

# Dataset大部分时候都够用，但是也支持pandas等底层操作

In [ ]:
dataset.set_format('pandas') # 转为pandas格式

In [133]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [134]:
dataset['train'][:3]

,patient_id,drugName,condition,review,rating,date,usefulCount,review_length
0,95260,Guanfacine,adhd,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,141
1,92703,Lybrel,birth control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,134
2,138000,Ortho Evra,birth control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,89


In [ ]:
train_df = dataset['train'][:]
# 实际上， Dataset.set_format() 仅仅改变了数据集的 __getitem__() 方法的返回格式。这意味着当我们想从 "pandas" 格式的 Dataset 中创建像 train_df 这样的新对象时，我们需要对整个数据集进行切片（[:]）才可以获得 pandas.DataFrame 对象。无论输出格式如何，你都可以自己验证 drug_dataset["train"] 的类型依然还是 Dataset 

In [146]:
# 计算数据分布
frequencies =  train_df['condition'].value_counts().to_frame().reset_index().rename(columns={"index": "condition", "count": "frequency"})

frequencies.head()

,condition,frequency
0,birth control,27655
1,depression,8023
2,acne,5209
3,anxiety,4991
4,pain,4744


In [147]:
# 从pandas创建Dataset对象
from datasets import Dataset

freq_dataset = Dataset.from_pandas(frequencies)
freq_dataset

Dataset({
    features: ['condition', 'frequency'],
    num_rows: 819
})

In [ ]:
# Dataset对象转为pandas后可以再转回去
dataset.reset_format()
dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 138514
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

In [ ]:
# 创建随机验证集
data_set_clean = dataset['train'].train_test_split(train_size=0.8,seed=42) # 得到了80%的数据
# 重命名 test集合作为验证集合
data_set_clean['validation'] = data_set_clean.pop('test')
# 原集合的test作为test 一个总的验证集合
data_set_clean['test'] = dataset['test']
data_set_clean

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

In [ ]:
# 保存数据集合  格式是arrow，将 Arrow 格式视为一个优化过的列和行的精美表格，它针对构建处理和传输大型数据集的高性能应用程序进行了优化。
data_set_clean.save_to_disk("./drug-reviews")

Saving the dataset (0/1 shards):   0%|          | 0/110811 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/27703 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/46108 [00:00<?, ? examples/s]

In [153]:
# 读取集合
from datasets import load_from_disk

drug_dataset_reloaded = load_from_disk('drug-reviews')
drug_dataset_reloaded

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

In [157]:
# 保存为json格式和加载

for part,dataset in data_set_clean.items():
    dataset.to_json(f"./drug-view-json/drug-review-{part}.jsonl")

Creating json from Arrow format:   0%|          | 0/111 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/28 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/47 [00:00<?, ?ba/s]

In [159]:
# json格式加载
from datasets import load_dataset
data_set_clean = load_dataset(
    "json"
    ,data_files={
        "train":"./drug-view-json/drug-review-train.jsonl"
        ,"validation":"./drug-view-json/drug-review-validation.jsonl"
        ,"test":"./drug-view-json/drug-review-test.jsonl"
        }
    )
data_set_clean

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})